<a href="https://www.kaggle.com/code/faiqueali/in-progress-data-preprocessing?scriptVersionId=141812067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv


# Imports

In [2]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# Constants

In [3]:
DATASET_PATH = '/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv'
REGEX_MOVIE_YEAR = r'\(\d{4}\)|\([MDCLXVI]+(?:M{0,3}D?C{0,3}L?X{0,3}V?I{0,3})\)\(\d{4}\)|\([MDCLXVI]+(?:M{0,3}D?C{0,3}L?X{0,3}V?I{0,3})\)'

# Gather Data

In [4]:
# Load the dataset
data = pd.read_csv(DATASET_PATH)

# Show first 10 values
data.head(10)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN
5,Outer Banks,(2020– ),"\nAction, Crime, Drama",7.6,\nA group of teenagers from the wrong side of ...,"\n \n Stars:\nChase Stokes, \nMa...","25,858",50.0,NaN
6,The Last Letter from Your Lover,(2021),"\nDrama, Romance",6.8,\nA pair of interwoven stories set in the past...,\n Director:\nAugustine Frizzell\n| \n S...,"5,283",110.0,NaN
7,Dexter,(2006–2013),"\nCrime, Drama, Mystery",8.6,"\nBy day, mild-mannered Dexter is a blood-spat...","\n \n Stars:\nMichael C. Hall, \...","665,387",53.0,NaN
8,Never Have I Ever,(2020– ),\nComedy,7.9,\nThe complicated life of a modern-day first g...,\n \n Stars:\nMaitreyi Ramakrish...,"34,530",30.0,NaN
9,Virgin River,(2019– ),"\nDrama, Romance",7.4,"\nSeeking a fresh start, nurse practitioner Me...",\n \n Stars:\nAlexandra Breckenr...,"27,279",44.0,NaN


In [5]:
data.shape

(9999, 9)

#### Check for null values in dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [7]:
data.isna().sum().to_frame('Null Values Count')

,Null Values Count
MOVIES,0
YEAR,644
GENRE,80
RATING,1820
ONE-LINE,0
STARS,0
VOTES,1820
RunTime,2958
Gross,9539


#### Check for duplicates

In [8]:
data.duplicated().sum()

431

In [9]:
# Dropping the duplication
data.drop_duplicates(inplace=True)
data.shape

(9568, 9)

In [10]:
data.isna().sum().to_frame('Null Values Count')

,Null Values Count
MOVIES,0
YEAR,542
GENRE,78
RATING,1400
ONE-LINE,0
STARS,0
VOTES,1400
RunTime,2560
Gross,9108


# Clean Data

In [11]:
# Remove unnecessary attributes
remove_columns = ['ONE-LINE', 'STARS']
data = data.drop(columns=remove_columns, axis=1)
data.head(5)

,MOVIES,YEAR,GENRE,RATING,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,"17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,"885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,"414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,NaN,NaN,NaN


In [12]:
# Remove rows where YEAR is NaN
data = data.dropna(subset=["YEAR"])
data.shape

(9026, 7)

# Data Transformation

In [13]:
# Create a conditional column "Type"
data['Type'] = data['YEAR'].apply(lambda x: 'Movie' if pd.notna(x) and pd.Series(x).str.match(REGEX_MOVIE_YEAR).any() else 'Series')
data.head(5)

,MOVIES,YEAR,GENRE,RATING,VOTES,RunTime,Gross,Type
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,"21,062",121.0,NaN,Movie
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,"17,870",25.0,NaN,Series
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,"885,805",44.0,NaN,Series
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,"414,849",23.0,NaN,Series
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,NaN,NaN,NaN,Movie


In [14]:
# Fill missing values
data['GENRE'].fillna('None', inplace=True)
data['RATING'].fillna(0.0, inplace=True)
data['VOTES'].fillna('0', inplace=True)
data['RunTime'].fillna(0, inplace=True)
data['Gross'].fillna(0, inplace=True)

In [15]:
data.isna().sum().to_frame('Null Values Count')

,Null Values Count
MOVIES,0
YEAR,0
GENRE,0
RATING,0
VOTES,0
RunTime,0
Gross,0
Type,0


In [16]:
# Check unique values
data.nunique()

MOVIES     6355
YEAR        438
GENRE       505
RATING       83
VOTES      4130
RunTime     262
Gross       333
Type          2
dtype: int64

# Check correlation and covariance

In [17]:
df_insights = data[['RATING', 'Gross']]